In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
plt.close('all')

/Users/khw/.pyenv/versions/3.9.1/envs/assembly/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
frames = []
import traceback
for txt in os.listdir('./aol')[0:1]:
    try:
        queries = pd.read_csv(f'./aol/{txt}', sep='\t', dtype={"AnonID": "Int64", "Query": "string", "QueryTime": "string", "ItemRank": "Int32", "ClickURL": "string"}, keep_default_na=False, na_values=[""])
        frames.append(queries)
    except Exception as e:
        print(txt)
        print(e)
        print(e.args)
        traceback.print_exc()

queries = pd.concat(frames)
queries['QueryTime'] = pd.to_datetime(queries['QueryTime'])
queries.insert(5, 'SessionNum', 0)
queries.insert(6, 'Type', '')

In [3]:
grouped_by_id = queries.groupby('AnonID')
qqq = grouped_by_id['Query'].nunique()
print(len(grouped_by_id))
ppp = qqq.sort_values(ascending=False)

filtered = ppp.index


65516


In [4]:
def dict_from_row(row, logType, sessionNum):
    return {
      'AnonID': row['AnonID'],
      'Query': row['Query'],
      'QueryTime': row['QueryTime'],
      'Type': logType,
      'SessionNum': sessionNum,
      'ItemRank': row['ItemRank'] if logType == "Click" else np.nan,
      'ClickURL': row['ClickURL'] if logType == "Click" else np.nan
    }

# print(queries)

print(filtered)

new_logs = []
for id in filtered:
    user_log = queries.loc[queries['AnonID'] == id]
    current_query = ''
    current_query_time = ''
    current_session_num = 0
    for i in range(len(user_log)):
        if user_log.iat[i, 2] >= current_query_time + pd.Timedelta(30, unit="min"):
            current_session_num += 1
            current_query = user_log.iat[i, 1]
            current_query_time = user_log.iat[i, 2]

            new_logs.append(dict_from_row(user_log.iloc[i], "Query", current_session_num))

            if pd.notna(user_log.iat[i, 4]):
                new_logs.append(dict_from_row(user_log.iloc[i], "Click", current_session_num))

        else:
            # Within the same sessoin
            if (user_log.iat[i, 1] == current_query) and (user_log.iat[i, 2] > current_query_time):
                # pagination case
                current_query_time = user_log.iat[i, 2]
                new_logs.append(dict_from_row(user_log.iloc[i], "NextPage", current_session_num))

                if pd.notna(user_log.iat[i, 4]):
                    new_logs.append(dict_from_row(user_log.iloc[i], "Click", current_session_num))
            elif (user_log.iat[i, 1] == current_query) and (user_log.iat[i, 2] == current_query_time):
                # Same query, same timestamp ==> must be different clicks
                if pd.notna(user_log.iat[i, 4]):
                    new_logs.append(dict_from_row(user_log.iloc[i], "Click", current_session_num))
                else:
                    # Another very quick nextpage
                    new_logs.append(dict_from_row(user_log.iloc[i], "NextPage", current_session_num))
            elif (user_log.iat[i, 1] != current_query) :
                ## Query refinement
                current_query_time = user_log.iat[i, 2]
                current_query = user_log.iat[i, 1]
                new_logs.append(dict_from_row(user_log.iloc[i], "Query", current_session_num))

                if pd.notna(user_log.iat[i, 4]):
                    ## If user clicked sth from query refinement
                    new_logs.append(dict_from_row(user_log.iloc[i], "Click", current_session_num))
        


new_logs_df = pd.DataFrame(new_logs)
new_logs_df.to_csv('./new_logs.csv', index_label="idx")


Index([   91602,  2846780,  1750999,   118401,   186465,  1496938,   843043,
         929313,   130832,  3162221,
       ...
       17033862,  5125765, 23199592, 17040189, 23199322, 23199295, 10703368,
       19387051,  5124820, 23219274],
      dtype='object', name='AnonID', length=65516)
